In [1]:
from bs4 import BeautifulSoup
import pymongo
import requests
import re
import sys

In [2]:
# Create mongodb connection
from pymongo import MongoClient
connection = MongoClient('localhost', 27017)
database = connection.housing_data

table = database.etuovi

#table.drop() to drop the collection 'housing_data'

cookie = {'org.springframework.mobile.device.site.CookieSitePreferenceRepository.SITE_PREFERENCE':'NORMAL','JSESSIONID':'183EE2F12C128B603790A0D5D1558022','evid_0006':'87ccc59d-4b80-489f-98cf-be92d6591fa6','evid_0006_ref':'false','adptseg_0006':'kv1001#a-kv1002#c-kv1003#d-kv1004#d-kv1005#c-kv1006#a:b-kv1008#c-kv1009#17-kv1010#b-kv1011#f-kv1012#c-kv1013#a-kv1016#1:11:13:2:7-kv1017#2-','adptset_0006':'1','evid_0006_set':'2','__utma':'82691985.304271103.1385626909.1385642039.1385658686.4','__utmb':'82691985.9.8.1385658723878','__utmc':'82691985','__utmz':'82691985.1385626909.1.1.utmcsr=kuluttaja.etuovi.com|utmccn=(referral)|utmcmd=referral|utmcct=/crometapp/product/realties/common/public/frontpage2/search_in_progress_to_vuokraovi.jsp','slbp_80':'2265258762.20480.0000','slbp_443':'2265258762.47873.0000'}

data_added = 0

In [3]:
start_page = 1
end_page = 520
Links = []
for page in range(start_page, end_page):
    print ('Parsing page:', str(page))
    html = requests.post('http://www.vuokraovi.com/vuokra-asunnot?page=' + str(page), cookies=cookie, data={'listorder':11})
    
    parsed_html = BeautifulSoup(html.text,'html.parser')                                                                                        
    #print(parsed_html.prettify())
    url_tag = parsed_html.find_all('a', {'class':'list-item-link'})
    for link in url_tag:
        address = link.get('href')
        if '/vuokra-asunto' in address:
            #print(address)
            Links.append(address)
#print (Links)

Parsing page: 1
Parsing page: 2
Parsing page: 3
Parsing page: 4
Parsing page: 5
Parsing page: 6
Parsing page: 7
Parsing page: 8
Parsing page: 9
Parsing page: 10
Parsing page: 11
Parsing page: 12
Parsing page: 13
Parsing page: 14
Parsing page: 15
Parsing page: 16
Parsing page: 17
Parsing page: 18
Parsing page: 19
Parsing page: 20
Parsing page: 21
Parsing page: 22
Parsing page: 23
Parsing page: 24
Parsing page: 25
Parsing page: 26
Parsing page: 27
Parsing page: 28
Parsing page: 29
Parsing page: 30
Parsing page: 31
Parsing page: 32
Parsing page: 33
Parsing page: 34
Parsing page: 35
Parsing page: 36
Parsing page: 37
Parsing page: 38
Parsing page: 39
Parsing page: 40
Parsing page: 41
Parsing page: 42
Parsing page: 43
Parsing page: 44
Parsing page: 45
Parsing page: 46
Parsing page: 47
Parsing page: 48
Parsing page: 49
Parsing page: 50
Parsing page: 51
Parsing page: 52
Parsing page: 53
Parsing page: 54
Parsing page: 55
Parsing page: 56
Parsing page: 57
Parsing page: 58
Parsing page: 59
Parsin

Parsing page: 463
Parsing page: 464
Parsing page: 465
Parsing page: 466
Parsing page: 467
Parsing page: 468
Parsing page: 469
Parsing page: 470
Parsing page: 471
Parsing page: 472
Parsing page: 473
Parsing page: 474
Parsing page: 475
Parsing page: 476
Parsing page: 477
Parsing page: 478
Parsing page: 479
Parsing page: 480
Parsing page: 481
Parsing page: 482
Parsing page: 483
Parsing page: 484
Parsing page: 485
Parsing page: 486
Parsing page: 487
Parsing page: 488
Parsing page: 489
Parsing page: 490
Parsing page: 491
Parsing page: 492
Parsing page: 493
Parsing page: 494
Parsing page: 495
Parsing page: 496
Parsing page: 497
Parsing page: 498
Parsing page: 499
Parsing page: 500
Parsing page: 501
Parsing page: 502
Parsing page: 503
Parsing page: 504
Parsing page: 505
Parsing page: 506
Parsing page: 507
Parsing page: 508
Parsing page: 509
Parsing page: 510
Parsing page: 511
Parsing page: 512
Parsing page: 513
Parsing page: 514
Parsing page: 515
Parsing page: 516
Parsing page: 517
Parsing pa

In [4]:
allApartments = []
apartment = dict()
for page in Links:
    #print(page)
    html = requests.post('http://www.vuokraovi.com' + page, cookies=cookie, data={'listorder':11})
    parsed_html = BeautifulSoup(html.text,'html.parser') 
    #print(parsed_html)
    test_tagth = parsed_html.find_all('th')
    test_tagtd = parsed_html.find_all('td')
    apartment['postal_code'] = str(test_tagtd[0].find('span', {'itemprop':'addressLocality'}).get_text(strip = True)[:5])
  
    
    for i in range(0,len(test_tagth)):
        if 'Vakuus:' in test_tagth[i].get_text():
            apartment['deposit'] = test_tagtd[i].get_text(strip = True)
            
        if 'Vuokra:' in test_tagth[i].get_text():
            apartment['rent'] = test_tagtd[i].get_text(strip = True) 
            
        if 'Tyyppi:' in test_tagth[i].get_text():
            apartment['type'] = test_tagtd[i].get_text(strip = True)
            
        if 'Huoneiden lukumäärä' in test_tagth[i].get_text():
            apartment['number_of_rooms'] = test_tagtd[i].get_text(strip = True)
            
        if 'Rakennusvuosi:' in test_tagth[i].get_text():
            apartment['built_year'] = test_tagtd[i].get_text(strip = True)
            
        if 'Yleiskunto:' in test_tagth[i].get_text():
            apartment['condition'] = test_tagtd[i].get_text(strip = True)
            
        if 'sallittu:' in test_tagth[i].get_text():
            apartment['allow_pets'] = test_tagtd[i].get_text(strip = True)
            
        if 'pinta-ala:' in test_tagth[i].get_text():
            apartment['total_size'] = test_tagtd[i].get_text(strip = True)
    
    allApartments.append(apartment)
    
    try:
        # Push the collected data to db.
        table.insert({
                'postal_code': apartment['postal_code'],
                'built_year': apartment['built_year'], 
                'type': apartment['type'],
                'total_size': apartment['total_size'],
                'number_of_rooms': apartment['number_of_rooms'],
                'condition': apartment['condition'],
                'rent': apartment['rent'],
                'deposit': apartment['deposit'],
                'allow_pets': apartment['allow_pets']          
            }) 
        data_added += 1
    except pymongo.errors.PyMongoError:
        pass


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


IndexError: list index out of range

In [6]:
print (len(allApartments))
print (data_added)

561
561


In [ ]:
import pandas
print (pandas.DataFrame(allApartments))

In [ ]:
# Make a query to the specific DB and Collection
cursor = database.etuovi.find()

# Expand the cursor and construct the DataFrame
df =  pd.DataFrame(list(cursor))
del df['_id']
del df['condition']
del df['deposit']
del df['allow_pets']
print (df)